In [11]:
import tensorflow as tf
from tensorflow.contrib import rnn
import pandas as pd
import numpy as np

class DataSrc(object):
    def __init__(self, seqlen=10):
        history = pd.read_excel('history.xlsx')
        del history['Time']
        history = (history - history.mean()) / (history.max() - history.min())

        history = history.as_matrix()
        predict = pd.read_excel('predict.xlsx').as_matrix()[:, 1:]
        self.data = np.array([history[0:seqlen, :]])
        self.labels = np.array([predict[0:seqlen, :]])
        for i in range(1, len(history) - seqlen):
            self.data = np.vstack((self.data, [history[i:i + seqlen, :]]))
            self.labels = np.vstack((self.labels, [predict[i:i + seqlen, :]]))
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels

In [12]:
ds = DataSrc()

In [13]:
ds.data.shape

(4719, 10, 14)

In [14]:
ds.labels.shape

(4719, 10, 1)